<a href="https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject_person_reid_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject.ipynb](https://colab.research.google.com/github/dadebulba/DeepLearningProject/blob/main/DeepLearningProject.ipynb)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [2]:
!unzip "/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/dataset.zip" -d dataset

Output streaming troncato alle ultime 5000 righe.
  inflating: dataset/train/0350_c1_083624030.jpg  
  inflating: dataset/train/1411_c1_004276123.jpg  
  inflating: dataset/train/1411_c1_063612070.jpg  
  inflating: dataset/train/1411_c3_073654330.jpg  
  inflating: dataset/train/1411_c5_065380551.jpg  
  inflating: dataset/train/0959_c1_091954569.jpg  
  inflating: dataset/train/0959_c3_014933988.jpg  
  inflating: dataset/train/0959_c5_006154919.jpg  
  inflating: dataset/train/0959_c6_045481865.jpg  
  inflating: dataset/train/0852_c6_004791953.jpg  
  inflating: dataset/train/1177_c3_011636446.jpg  
  inflating: dataset/train/0474_c1_043109826.jpg  
  inflating: dataset/train/0474_c5_069409220.jpg  
  inflating: dataset/train/0474_c5_013563167.jpg  
  inflating: dataset/train/0474_c5_029799244.jpg  
  inflating: dataset/train/1487_c1_003967228.jpg  
  inflating: dataset/train/1487_c5_046010440.jpg  
  inflating: dataset/train/1487_c5_044022610.jpg  
  inflating: dataset/train/1487_

# Deep Learning Project - People ReID

In [3]:
# import necessary libraries
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import os
from os import listdir
from os.path import isfile, join
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import random
random.seed(10)
# print cuda info
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Cuda device count: {torch.cuda.device_count()}")

Cuda available: True
Cuda device count: 1


# Network
## Siamese Network

In [6]:
 '''
Input arguments
  num_classes: number of classes in the dataset.
               This is equal to the number of output neurons.
'''

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

class Siamese(nn.Module):

    def __init__(self, resnet):
        super(Siamese, self).__init__()
        self.resnet = resnet
        self.resnet.fc = Identity()
        self.linear = torch.nn.Sequential(
          torch.nn.Linear(in_features=2048, out_features=1024),
          torch.nn.Linear(in_features=1024, out_features=512),
          torch.nn.Sigmoid()
        )

    def forward_one(self, x):
        x = self.resnet(x)
        x = x.view(x.size()[0], -1)
        x = self.linear(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        return out1, out2

def initialize_alexnet(num_classes):
  # load the pre-trained Alexnet
  #alexnet = torchvision.models.alexnet(pretrained=True)
  wide_resnet = torchvision.models.resnet50(pretrained=True)
  num_features = wide_resnet.fc.in_features
  wide_resnet.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=num_features, out_features=1024),
    torch.nn.Linear(in_features=1024, out_features=512),
    torch.nn.Linear(in_features=512, out_features=num_classes),
    torch.nn.Sigmoid()
  )
  #print(resnext)
  # get the number of neurons in the penultimate layer
  #in_features = alexnet.classifier[6].in_features
  
  # re-initalize the output layer
  #alexnet.classifier[6] = torch.nn.Sequential(
  #  torch.nn.Linear(in_features=in_features, out_features=num_classes),
  #  torch.nn.Sigmoid()
  #)
  return wide_resnet

In [ ]:
class PeopleTestDataset(Dataset):
    """People with annotations dataset."""

    def __init__(self, X1, X2, query_dir, test_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X1 = X1
        self.X2 = X2
        self.query_dir = query_dir
        self.test_dir = test_dir

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name1 = self.X1[idx]
        img_name2 = self.X2[idx]

        image1 = Image.open("%s/%s" % (self.query_dir, img_name1))
        image2 = Image.open("%s/%s" % (self.test_dir, img_name2))
        image1 = T.ToTensor()(image1)
        image2 = T.ToTensor()(image2)
        image1 = F.interpolate(image1, size=128)  
        image2 = F.interpolate(image2, size=128)  

        sample = (image1, img_name1, image2, img_name2)
        return sample

In [5]:
def getDataToEvaluate(test_dir, query_dir):
    test_files = [f for f in listdir(test_dir)]
    query_files = [f for f in listdir(query_dir)]

    X1 = []
    X2 = []
    for query in query_files:
        for test in test_files:
            X1.append(query)
            X2.append(test)
    return X1, X2
    
def imshow(img,text=None,should_save=False):
    npimg = img.cpu().numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()   

In [28]:
def log_values(writer, step, loss, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)

def main(batch_size=128, 
         device='cuda:0', 
         learning_rate=0.001, 
         epochs=50, 
         img_root='./dataset',
         threshold=0.1):
  from torch.utils.tensorboard import SummaryWriter
  writer = SummaryWriter(log_dir="runs/exp4")

  # Instantiates the model
  net = initialize_alexnet(num_classes=56).to(device)
  net = Siamese(net)
  net.load_state_dict(torch.load("/content/drive/MyDrive/UNITN/5° anno/Deep Learning 2021/models/siamese_net_reid_resnet50_5epoch.pth"))
  net.to(device)
  X1, X2 = getDataToEvaluate(test_dir="dataset/test", query_dir="dataset/queries")
  print(len(X1), len(X2))
  test_data = PeopleTestDataset(X1=X1,
                                    X2=X2,
                                    query_dir="dataset/queries",
                                    test_dir="dataset/test")

  test_dataloader = torch.utils.data.DataLoader(test_data, 1, shuffle=False, num_workers=0) #before num_workers=4
    
  query_with_test = {}
  forward_test = {}
  forward_query = {}
  for x0, img_name0,x1, img_name1 in test_dataloader:
      x0 = x0.to('cuda:0')
      x1 = x1.to('cuda:0')

      # Preparo lista da printare su txt
      if img_name0 not in query_with_test:
          query_with_test[img_name0] = []

      # Forward query element
      if img_name0 not in forward_query:
          forward_query[img_name0] = net.forward_one(x0)

      # Forward test element
      if img_name1 not in forward_test:
          forward_test[img_name1] = net.forward_one(x1)

      euclidean_distance = F.pairwise_distance(forward_query[img_name0], forward_test[img_name1])
      print(euclidean_distance.item() < threshold)
      if euclidean_distance.item() < threshold:
          query_with_test[img_name0].append(img_name1)
          if (len(query_with_test[img_name0]) % 100 == 0):
                print(len(query_with_test[img_name0]))
          concatenated = torch.cat((x0,x1),0)
          #print(i, euclidean_distance)
          imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))
      # Free GPU memory
      torch.cuda.empty_cache()
      import gc
      gc.collect()
  print(query_with_test)

In [29]:
main()

44238392 44238392
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


KeyboardInterrupt: ignored